In [10]:
from lxml import etree
import re
import math
import numpy as np
import pandas as pd
from pprint import pprint
from time import time
from sklearn import metrics
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.cluster import DBSCAN
from sklearn.decomposition import TruncatedSVD, PCA
from sklearn.preprocessing import Normalizer
from sklearn.pipeline import Pipeline
from sklearn.mixture import BayesianGaussianMixture
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from matplotlib import pyplot as plt
from sklearn.cluster import KMeans, MiniBatchKMeans
%matplotlib inline

In [24]:
class Text(BaseEstimator, TransformerMixin): 
    def __init__(self, lenMin=2000, lenMax=10000, chunks=False): 
        self.lenMin = lenMin
        self.lenMax = lenMax
        self.chunks=chunks
        
    def fit(self, *_):
        print('heyo! fitting')
        return self

    def transform(self, filename): 
        print('heyo! transforming')
        lenMin, lenMax = self.lenMin, self.lenMax
        self.tree = etree.parse(filename)
        self.allSaidElems = self.tree.findall('.//said[@who]')
        # Only get those in our length range
        self.saidElems = [elem for elem in self.allSaidElems if len(elem.text)>lenMin and len(elem.text)<lenMax]
        self.allChars = [elem.attrib['who'] for elem in self.saidElems]
        self.chars = list(set(self.allChars))
        self.labeledText = [(elem.attrib['who'], elem.text) for elem in self.saidElems]
        self.labeledText = [(item[0], self.clean(item[1])) for item in self.labeledText]
        self.labels = [item[0] for item in self.labeledText]
        charDict = {'Bernard': 0, 'Louis': 1, 'Neville': 2,
                'Rhoda': 3, 'Jinny': 4, 'Susan': 5}
        self.numericLabels = [charDict[label] for label in self.labels]
        self.allText = [item[1] for item in self.labeledText]
        self.charDict = self.makeCharDict()
        self.charChunks, self.charChunksLabels = self.makeCharChunks()
        if self.chunks: 
            self.allText = self.charChunks
            self.labels = self.charChunksLabels
        self.lengths = [len(item) for item in self.allText]
        return self.allText
        
    def makeCharDict(self): 
        """ Make a dictionary of each character's total speech. """
        # Initialize empty dictionary. 
        charDict = {char: "" for char in self.chars}
        for elem in self.allSaidElems: 
            charDict[elem.attrib['who']]+=elem.text
        return charDict
            
    def makeCharChunks(self, n=2): 
        """ Make a list of chunks of character speech. """
        charChunks = []
        charChunksLabels = []
        for char, text in self.charDict.items(): 
            #chunks = self.chunkText(text, n)
            chunks = self.sliceText(text)
            for i, chunk in enumerate(chunks): 
                charChunks.append(chunk)
                charChunksLabels.append(char + '-%s' % i)
        return charChunks, charChunksLabels
        
    def chunkText(self, text, n=2): 
        """ Breaks one text into n texts."""
        newLen = math.floor(len(text) / n)
        parts = [text[i:i+newLen] for i in range(0, len(text), newLen)]
        if len(parts) > n: 
            parts[-2]+=parts[-1]
            parts = parts[:n]
        return parts
    
    def sliceText(self, text, size=8000):
        parts = []
        while len(text) > size: 
            part = text[:size]
            text = text[size:]
            parts.append(part)
        return parts

    def clean(self, utterance): 
        """ 
        Cleans utterances. 
        """
        # Remove "said Bernard," etc. 
        charRegex = "said (%s)," % '|'.join(self.chars)
        out = re.sub(charRegex, '', utterance)
       
        # Remove quotation marks. 
        out = re.sub('[“”"]', '', out)
        
        # Remove line breaks. 
        out = re.sub('\n', ' ', out)
        return out

In [25]:
# Code adapted from http://stackoverflow.com/a/28384887/584121
class DenseTransformer(BaseEstimator, TransformerMixin):
    
    def __init__(self): 
        return

    def transform(self, X, y=None, **fit_params):
        return X.todense()

    def fit_transform(self, X, y=None, **fit_params):
        self.fit(X, y, **fit_params)
        return self.transform(X)

    def fit(self, X, y=None, **fit_params):
        return self

In [45]:
parameters = { 'tfidf__use_idf': (True, False), 
               'tfidf__min_df': (0.0, 0.3, 0.4),
               'tfidf__max_df': (0.5, 0.7, 1.0),
               'tfidf__max_features': (100, 200, 500, 1000, 2000), 
               'pca__n_components': (2, 5, 10, 20, 25, 30, 50),
               'bgm__n_components': (2, 4, 6)}

In [46]:
text = Text().fit()
docs = text.transform('waves-tei.xml')
labels = text.numericLabels

heyo! fitting
heyo! transforming


In [47]:
pipeline = Pipeline([  
#                          ('text', Text()), 
                         ('tfidf', TfidfVectorizer()),
                         ('todense', DenseTransformer()),
                         ('pca', PCA(n_components=10)),
                         ('bgm', BayesianGaussianMixture(n_components=6)),
                        ])

In [48]:
grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1, verbose=1)

print("Performing grid search...")
print("pipeline:", [name for name, _ in pipeline.steps])
print("parameters:")
pprint(parameters)
t0 = time()
grid_search.fit(docs, labels)
print("done in %0.3fs" % (time() - t0))
print()

print("Best score: %0.3f" % grid_search.best_score_)
print("Best parameters set:")
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

Performing grid search...
pipeline: ['tfidf', 'todense', 'pca', 'bgm']
parameters:
{'bgm__n_components': (2, 4, 6),
 'pca__n_components': (2, 5, 10, 20, 25, 30, 50),
 'tfidf__max_df': (0.5, 0.7, 1.0),
 'tfidf__max_features': (100, 200, 500, 1000, 2000),
 'tfidf__min_df': (0.0, 0.3, 0.4),
 'tfidf__use_idf': (True, False)}
Fitting 3 folds for each of 1890 candidates, totalling 5670 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    2.9s
/usr/lib/python3.5/site-packages/sklearn/mixture/base.py:237: ConvergenceWarning: Initialization 1 did not converged. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  % (init + 1), ConvergenceWarning)
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   14.3s
/usr/lib/python3.5/site-packages/sklearn/mixture/base.py:237: ConvergenceWarning: Initialization 1 did not converged. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  % (init + 1), ConvergenceWarning)
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:   29.1s
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 1242 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done 1792 tasks      | elapsed: 89.1min
/usr/lib/python3.5/site-packages/sklearn/mixture/base.py:237: ConvergenceWarning: Initialization 1 did not converged. Try different init paramet

done in 5586.093s

Best score: 29.903
Best parameters set:
	bgm__n_components: 2
	pca__n_components: 25
	tfidf__max_df: 1.0
	tfidf__max_features: 500
	tfidf__min_df: 0.0
	tfidf__use_idf: True


[Parallel(n_jobs=-1)]: Done 5670 out of 5670 | elapsed: 93.1min finished
